# Azure AI Agent Service - Agent Creation (Programmatic)

Notebook contains logic for creating three agents (Analyst, Reviewer, Formatter) which will be used in coordination for analyzing complex documents. Users will have to opportunity to further edit these agent instructions either directly via the Azure AI Foundry portal, or from a provided UI with editable instruction windows.

Note: You can also manually create these agents in the Foundry UI and capture the IDs

In [3]:
import json

import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import MessageTextContent
from azure.ai.projects.models import CodeInterpreterTool, MessageAttachment
from azure.ai.projects.models import FilePurpose
from pathlib import Path
from opentelemetry import trace
from azure.monitor.opentelemetry import configure_azure_monitor
import base64

from azure.ai.projects.models import (
    AgentEventHandler,
    FunctionTool,
    MessageDeltaChunk,
    RequiredFunctionToolCall,
    RunStep,
    RunStepStatus,
    SubmitToolOutputsAction,
    ThreadMessage,
    ThreadRun,
    ToolOutput,
    BingGroundingTool
)
from typing import Any
from dotenv import load_dotenv

load_dotenv('../.env', override=True)

True

### Create connection to Azure AI Foundry

In [4]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ['AZURE_AI_FOUNDRY_CONNECTION_STRING'],
)

### Define and Create Agents

Note: after creation, you can reference these agents by ID in your FastAPI app

In [ ]:
analyst_agent = project_client.agents.create_agent(
    model="gpt-4o",
    name="cda-analyst-agent",
    instructions=f"""You are an AI assistant responsible for extracting targeted information from complex documents.

You will receive scanned images of the document along with an OCR extract of diagram text provided by Azure Document Intelligence in markdown format.

Your task is to review the provided information and create a JSON extract adhering to the schema provided below.

You may be asked to refine an existing extract based on feedback from an independent reviewer. In that case, incorporate those suggestions into your final extract.

# RULES:
1. When completing the extraction template, you **must** use text from the provided OCR results to avoid transcription errors.  
2. Your final response should consist of **only** JSON matching the template schema below.""",
    temperature=0.0,
    headers={"x-ms-enable-preview": "true"}
)

reviewer_agent = project_client.agents.create_agent(
    model="gpt-4o",
    name="cda-reviewer-agent",
    instructions='''You are an AI assistant responsible for reviewing and approving the extraction of targeted information from complex documents.

You will be provided with:
- A JSON extract of targeted information
- Images of the document
- A raw OCR extract of the product diagram (in Markdown format)
- Key-value pairs provided by Azure Document Intelligence

The template below shows the schema of the JSON extract you will receive for review.

Your task is to review the provided extract alongside the image and OCR data to determine whether the information has been extracted correctly.

- If the extraction is complete and accurate, set the `"complete"` field to `TRUE`.
- If the only errors in the extraction are on fields listed in **Reviewed Fields**, also set the `"complete"` field to `TRUE`.
- If the extraction is incomplete or contains inaccuracies for fields *not* listed in **Reviewed Fields**, you must provide feedback on the specific fields that need correction.

# RULES:
1. Any missing fields must have `null` in their place.  
2. Use the OCR text to verify the extracted values, ensuring that characters are correct.  
3. If the extraction is complete, set `"complete"` to `TRUE`.  
4. **Do not provide feedback** on fields listed in the **Reviewed Fields** list.  
5. You must provide your rationale describing why you gave the feedback, ensuring there is no conflicting information.


Your final response should be **only a single JSON object** consisting of the structure below:

```json
{
  "complete": <TRUE or FALSE>,
  "errors": [
    {
      "field": "<FIELD_NAME>",
      "message": "<ERROR_MESSAGE>"
    }
    // ... Repeat as necessary for other erroneous fields
  ],
  "rationale": "<YOUR_REASONING_FOR_THE_FEEDBACK_ABOVE>"
}
```
''',
    temperature=0.0,
    headers={"x-ms-enable-preview": "true"}
)

formatter_agent = project_client.agents.create_agent(
    model="gpt-4o",
    name="cda-formatter-agent",
    instructions=f"""You are an AI assistant who is responsible for ensuring a data extract is appropriately formatted according to the conventions provided to you.

Below, you will be provided with the current document extract, along with an expected template including data types and examples.

Update and return the current extract ensuring that all data is formatted according to the expected template.

Your final response should be **only a single JSON object**.""",
    temperature=0.0,
    headers={"x-ms-enable-preview": "true"}
)

### Update `.env` file

In [ ]:
env_lines = open('../.env').readlines()

env_lines.append(f"ANALYST_AGENT_ID='{analyst_agent.id}'\n")
env_lines.append(f"REVIEWER_AGENT_ID='{reviewer_agent.id}'\n")    
env_lines.append(f"FORMATTER_AGENT_ID='{formatter_agent.id}'\n")
with open('../.env', 'w') as f:
    f.writelines(env_lines)

: 